# interpolation to healpix

**Note**: Use `pixi run jupyterlab` to start a jupyterlab instance from within the project.

The UTM tile comes with a affine transform, which we can use to simplify the transformation.

For others, like `conditions/geometry` which doesn't have an affine transform, and `conditions/meteorology`, which is not UTM, we'll have to use a different approach (potentially just nearest neighbour interpolation while keeping the original coordinates

**Note**: To use this notebook on JupyterHub with a Pangeo Docker image (e.g., pangeo-eosc), run the commands below and then reload your web browser. It may take a long time—this is normal. On pangeo-eosc, once you complete this setup, it will work automatically in future sessions because the installation persists in your $HOME directory.
```
wget -qO- https://pixi.sh/install.sh | sh
export PATH="$HOME/.pixi/bin/:$PATH"
git clone https://github.com/keewis/legacy-converters.git
cd legacy-converters/
git branch -av
git switch interpolation
pixi shell -e dev
python -m ipykernel install --user --name=legacy-converters
```


In [1]:
import os
os.environ["PROJ_DATA"]="/home/jovyan/legacy-converters/.pixi/envs/dev/share/proj"

In [2]:
import pystac_client
import xarray as xr
import xdggs

import legacy_converters

xr.set_options(keep_attrs=True, display_expand_attrs=False, use_opt_einsum=True)

In [3]:
from distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44013,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:37057,Total threads: 2
Dashboard: http://127.0.0.1:38703/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:45267,


## query the datasets

In [4]:
# Access cloud-optimized Sentinel-2 data via the EOPF STAC catalog
catalog = pystac_client.Client.open("https://stac.core.eopf.eodc.eu")

# Define oceanographic study area and time window
LON, LAT = -4.5, 48  # Bay of Biscay - known for consistent wave patterns
date = "2025-06-17/2025-06-17"

# Search criteria optimized for wave analysis

collection = "sentinel-2-l1c"
items = list(
    catalog.search(
        datetime=date,
        collections=[collection],
        intersects=dict(type="Point", coordinates=[LON, LAT]),
        query={
            "eo:cloud_cover": {
                "lt": 20
            },  # Cloud cover < 20% ensures clear ocean surface
            "view:sun_elevation": {
                "gt": 25
            },  # Filter for high sun elevation > 25° (→ sun zenith angle < 65°),
            # which places the sun near the zenith.
        },
    ).items()
)

for item in items:
    print(f"✅ {item.id}")

item = items[0]

✅ S2B_MSIL1C_20250617T112109_N0511_R037_T30UUU_20250617T131418


## open the dataset

In [5]:
dt = xr.open_datatree(
    item.assets["product"].href, engine="zarr", chunks={}, decode_timedelta=True
)
dt

<xarray.DataTree>
Group: /
│   Attributes: (3)
├── Group: /conditions
│   ├── Group: /conditions/geometry
│   │       Dimensions:                        (angle: 2, band: 13, y: 23, x: 23,
│   │                                           detector: 7)
│   │       Coordinates:
│   │         * angle                          (angle) <U7 56B 'zenith' 'azimuth'
│   │         * band                           (band) <U3 156B 'b01' 'b02' ... 'b11' 'b12'
│   │         * y                              (y) int64 184B 5400000 5395000 ... 5290000
│   │         * x                              (x) int64 184B 300000 305000 ... 410000
│   │         * detector                       (detector) int64 56B 3 4 5 6 7 8 9
│   │       Data variables:
│   │           mean_sun_angles                (angle) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
│   │           mean_viewing_incidence_angles  (band, angle) float64 208B dask.array<chunksize=(13, 2), meta=np.ndarray>
│   │           sun_angles                     (angle, y, x) float64 8kB dask.array<chunksize=(2, 23, 23), meta=np.ndarray>
│   │           viewing_incidence_angles       (band, detector, angle, y, x) float64 770kB dask.array<chunksize=(7, 4, 2, 23, 23), meta=np.ndarray>
│   ├── Group: /conditions/mask
│   │   ├── Group: /conditions/mask/detector_footprint
│   │   │   ├── Group: /conditions/mask/detector_footprint/r10m
│   │   │   │       Dimensions:  (y: 10980, x: 10980)
│   │   │   │       Coordinates:
│   │   │   │         * y        (y) int64 88kB 5399995 5399985 5399975 ... 5290225 5290215 5290205
│   │   │   │         * x        (x) int64 88kB 300005 300015 300025 300035 ... 409775 409785 409795
│   │   │   │       Data variables:
│   │   │   │           b02      (y, x) uint8 121MB dask.array<chunksize=(10980, 10980), meta=np.ndarray>
│   │   │   │           b03      (y, x) uint8 121MB dask.array<chunksize=(10980, 10980), meta=np.ndarray>
│   │   │   │           b04      (y, x) uint8 121MB dask.array<chunksize=(10980, 10980), meta=np.ndarray>
│   │   │   │           b08      (y, x) uint8 121MB dask.array<chunksize=(10980, 10980), meta=np.ndarray>
│   │   │   ├── Group: /conditions/mask/detector_footprint/r20m
│   │   │   │       Dimensions:  (y: 5490, x: 5490)
│   │   │   │       Coordinates:
│   │   │   │         * y        (y) int64 44kB 5399990 5399970 5399950 ... 5290250 5290230 5290210
│   │   │   │         * x        (x) int64 44kB 300010 300030 300050 300070 ... 409750 409770 409790
│   │   │   │       Data variables:
│   │   │   │           b05      (y, x) uint8 30MB dask.array<chunksize=(5490, 5490), meta=np.ndarray>
│   │   │   │           b06      (y, x) uint8 30MB dask.array<chunksize=(5490, 5490), meta=np.ndarray>
│   │   │   │           b07      (y, x) uint8 30MB dask.array<chunksize=(5490, 5490), meta=np.ndarray>
│   │   │   │           b11      (y, x) uint8 30MB dask.array<chunksize=(5490, 5490), meta=np.ndarray>
│   │   │   │           b12      (y, x) uint8 30MB dask.array<chunksize=(5490, 5490), meta=np.ndarray>
│   │   │   │           b8a      (y, x) uint8 30MB dask.array<chunksize=(5490, 5490), meta=np.ndarray>
│   │   │   └── Group: /conditions/mask/detector_footprint/r60m
│   │   │           Dimensions:  (y: 1830, x: 1830)
│   │   │           Coordinates:
│   │   │             * y        (y) int64 15kB 5399970 5399910 5399850 ... 5290350 5290290 5290230
│   │   │             * x        (x) int64 15kB 300030 300090 300150 300210 ... 409650 409710 409770
│   │   │           Data variables:
│   │   │               b01      (y, x) uint8 3MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│   │   │               b09      (y, x) uint8 3MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│   │   │               b10      (y, x) uint8 3MB dask.array<chunksize=(1830, 1830), meta=np.ndarray>
│   │   └── Group: /conditions/mask/l1c_classification
│   │       └── Group: /conditions/mask/l1c_classification/r60m
│   │               Dimensions:  (y: 1830, x: 18

## subset to a region of interest

Create raster indexes (from `rasterix`). This will allow us to keep the affine transform up-to-date, even after the subsetting.

In [6]:
raster_dt = dt.grid4earth.create_raster_indexes()

In [7]:
small_dt = raster_dt.sel(
    {
        "x": slice(
            dt["conditions/geometry/x"][0],
            dt["conditions/geometry/x"][1],
        ),
        "y": slice(
            dt["conditions/geometry/y"][0],
            dt["conditions/geometry/y"][1],
        ),
    }
)
small_dt

<xarray.DataTree>
Group: /
│   Attributes: (3)
├── Group: /conditions
│   ├── Group: /conditions/geometry
│   │       Dimensions:                        (angle: 2, band: 13, y: 2, x: 2, detector: 7)
│   │       Coordinates:
│   │         * angle                          (angle) <U7 56B 'zenith' 'azimuth'
│   │         * band                           (band) <U3 156B 'b01' 'b02' ... 'b11' 'b12'
│   │         * y                              (y) int64 16B 5400000 5395000
│   │         * x                              (x) int64 16B 300000 305000
│   │         * detector                       (detector) int64 56B 3 4 5 6 7 8 9
│   │       Data variables:
│   │           mean_sun_angles                (angle) float64 16B dask.array<chunksize=(2,), meta=np.ndarray>
│   │           mean_viewing_incidence_angles  (band, angle) float64 208B dask.array<chunksize=(13, 2), meta=np.ndarray>
│   │           sun_angles                     (angle, y, x) float64 64B dask.array<chunksize=(2, 2, 2), meta=np.ndarray>
│   │           viewing_incidence_angles       (band, detector, angle, y, x) float64 6kB dask.array<chunksize=(7, 4, 2, 2, 2), meta=np.ndarray>
│   ├── Group: /conditions/mask
│   │   ├── Group: /conditions/mask/detector_footprint
│   │   │   ├── Group: /conditions/mask/detector_footprint/r10m
│   │   │   │       Dimensions:  (y: 500, x: 500)
│   │   │   │       Coordinates:
│   │   │   │         * y        (y) float64 4kB 5.4e+06 5.4e+06 5.4e+06 ... 5.395e+06 5.395e+06
│   │   │   │         * x        (x) float64 4kB 3e+05 3e+05 3e+05 ... 3.05e+05 3.05e+05 3.05e+05
│   │   │   │       Data variables:
│   │   │   │           b02      (y, x) uint8 250kB dask.array<chunksize=(500, 500), meta=np.ndarray>
│   │   │   │           b03      (y, x) uint8 250kB dask.array<chunksize=(500, 500), meta=np.ndarray>
│   │   │   │           b04      (y, x) uint8 250kB dask.array<chunksize=(500, 500), meta=np.ndarray>
│   │   │   │           b08      (y, x) uint8 250kB dask.array<chunksize=(500, 500), meta=np.ndarray>
│   │   │   │       Indexes:
│   │   │   │         ┌ x        RasterIndex (crs=None)
│   │   │   │         └ y
│   │   │   ├── Group: /conditions/mask/detector_footprint/r20m
│   │   │   │       Dimensions:  (y: 250, x: 250)
│   │   │   │       Coordinates:
│   │   │   │         * y        (y) float64 2kB 5.4e+06 5.4e+06 5.4e+06 ... 5.395e+06 5.395e+06
│   │   │   │         * x        (x) float64 2kB 3e+05 3e+05 3e+05 ... 3.049e+05 3.05e+05 3.05e+05
│   │   │   │       Data variables:
│   │   │   │           b05      (y, x) uint8 62kB dask.array<chunksize=(250, 250), meta=np.ndarray>
│   │   │   │           b06      (y, x) uint8 62kB dask.array<chunksize=(250, 250), meta=np.ndarray>
│   │   │   │           b07      (y, x) uint8 62kB dask.array<chunksize=(250, 250), meta=np.ndarray>
│   │   │   │           b11      (y, x) uint8 62kB dask.array<chunksize=(250, 250), meta=np.ndarray>
│   │   │   │           b12      (y, x) uint8 62kB dask.array<chunksize=(250, 250), meta=np.ndarray>
│   │   │   │           b8a      (y, x) uint8 62kB dask.array<chunksize=(250, 250), meta=np.ndarray>
│   │   │   │       Indexes:
│   │   │   │         ┌ x        RasterIndex (crs=None)
│   │   │   │         └ y
│   │   │   └── Group: /conditions/mask/detector_footprint/r60m
│   │   │           Dimensions:  (y: 83, x: 83)
│   │   │           Coordinates:
│   │   │             * y        (y) float64 664B 5.4e+06 5.4e+06 5.4e+06 ... 5.395e+06 5.395e+06
│   │   │             * x        (x) float64 664B 3e+05 3.001e+05 3.001e+05 ... 3.049e+05 3.049e+05
│   │   │           Data variables:
│   │   │               b01      (y, x) uint8 7kB dask.array<chunksize=(83, 83), meta=np.ndarray>
│   │   │               b09      (y, x) uint8 7kB dask.array<chunksize=(83, 83), meta=np.ndarray>
│   │   │               b10      (y, x) uint8 7kB dask.array<chunksize=(83, 83), meta=np.ndarray>
│   │   │           Indexes:
│   │   │             ┌ x        RasterIndex (crs=None)
│   │

## Interpolate to healpix

Select the dataset to interpolate

In [8]:
ds = small_dt["/measurements/reflectance/r10m"].to_dataset()
ds

<xarray.Dataset> Size: 8MB
Dimensions:  (y: 500, x: 500)
Coordinates:
  * y        (y) float64 4kB 5.4e+06 5.4e+06 5.4e+06 ... 5.395e+06 5.395e+06
  * x        (x) float64 4kB 3e+05 3e+05 3e+05 ... 3.05e+05 3.05e+05 3.05e+05
Data variables:
    b02      (y, x) float64 2MB dask.array<chunksize=(500, 500), meta=np.ndarray>
    b03      (y, x) float64 2MB dask.array<chunksize=(500, 500), meta=np.ndarray>
    b04      (y, x) float64 2MB dask.array<chunksize=(500, 500), meta=np.ndarray>
    b08      (y, x) float64 2MB dask.array<chunksize=(500, 500), meta=np.ndarray>
Indexes:
  ┌ x        RasterIndex (crs=None)
  └ y

infer the target grid.

Note: the method on `grid4earth` handles ellipsoids, while `xdggs` does not (yet)

In [9]:
%%time
grid_info = xdggs.HealpixInfo(level=19, indexing_scheme="nested")
target_grid = ds.grid4earth.infer_healpix_grid(grid_info)
target_grid

CPU times: user 10.6 ms, sys: 4.72 ms, total: 15.3 ms
Wall time: 37.1 ms


<xarray.Dataset> Size: 1MB
Dimensions:   (cells: 162789)
Coordinates:
  * cell_ids  (cells) uint64 1MB 926644821885 926644821886 ... 926656528545
Dimensions without coordinates: cells
Data variables:
    *empty*
Indexes:
    cell_ids  HealpixIndex(level=19, indexing_scheme=nested, kind=pandas)

### Nearest neighbour interpolation

Compute interpolation weights

Note: this is very fast because we're using a functional representation of the source grid (a affine transform) to compute neighbours.

In [10]:
%%time
weights = legacy_converters.interpolation.weights.nearest_affine(ds, target_grid)
weights

CPU times: user 58 ms, sys: 11.9 ms, total: 69.9 ms
Wall time: 59.3 ms


Format,coo
Data Type,float16
Shape,"(162789, 500, 500)"
nnz,162789
Density,4e-06
Read-only,True
Size,4.0M
Storage ratio,0.00


Apply the interpolation with a sparse matrix multiplication:

In [11]:
%%time
regridded = ds.map(
    lambda var: xr.dot(
        weights.chunk({"cells": 2000000}).variable, var, dim=["x", "y"]
    ).assign_coords(weights.dggs.coord.coords)
)
regridded

CPU times: user 1.78 s, sys: 989 ms, total: 2.77 s
Wall time: 3.32 s


<xarray.Dataset> Size: 7MB
Dimensions:   (cells: 162789)
Coordinates:
  * cell_ids  (cells) uint64 1MB 926644821885 926644821886 ... 926656528545
Dimensions without coordinates: cells
Data variables:
    b02       (cells) float64 1MB dask.array<chunksize=(162789,), meta=sparse.numba_backend._coo.core.COO>
    b03       (cells) float64 1MB dask.array<chunksize=(162789,), meta=sparse.numba_backend._coo.core.COO>
    b04       (cells) float64 1MB dask.array<chunksize=(162789,), meta=sparse.numba_backend._coo.core.COO>
    b08       (cells) float64 1MB dask.array<chunksize=(162789,), meta=sparse.numba_backend._coo.core.COO>
Indexes:
    cell_ids  HealpixIndex(level=19, indexing_scheme=nested, kind=pandas)

Actually compute the results:

In [12]:
%%time
computed = regridded.compute().map(lambda var: var.copy(data=var.data.todense()))
computed

CPU times: user 178 ms, sys: 239 ms, total: 417 ms
Wall time: 3.77 s


<xarray.Dataset> Size: 7MB
Dimensions:   (cells: 162789)
Coordinates:
  * cell_ids  (cells) uint64 1MB 926644821885 926644821886 ... 926656528545
Dimensions without coordinates: cells
Data variables:
    b02       (cells) float64 1MB 0.221 0.2007 0.221 ... 0.281 0.2072 0.2226
    b03       (cells) float64 1MB 0.1888 0.1666 0.1888 ... 0.2525 0.1749 0.186
    b04       (cells) float64 1MB 0.1833 0.1538 0.1833 ... 0.2557 0.1637 0.1777
    b08       (cells) float64 1MB 0.169 0.1405 0.169 ... 0.2487 0.1502 0.1688
Indexes:
    cell_ids  HealpixIndex(level=19, indexing_scheme=nested, kind=pandas)

Display the result:

Note: `xdggs` does not yet support ellipsoidal coordinates, hence the shift compared to the base map

In [13]:
computed["b03"].dggs.explore(alpha=0.8)

Map(basemap=MaplibreBasemap(), controls=(FullscreenControl(), NavigationControl(), ScaleControl()), custom_att…

### Bilinear interpolation

Compute interpolation weights

Note: this is very fast because we're using a functional representation of the source grid (a affine transform) to compute neighbours.

Further improvements in the future:
- boundary treatment
- optimizations for sparse matrix-vector products

In [14]:
%%time
weights = legacy_converters.interpolation.weights.bilinear_affine(ds, target_grid)
weights

CPU times: user 71 ms, sys: 359 ms, total: 430 ms
Wall time: 381 ms


Format,coo
Data Type,float64
Shape,"(162789, 500, 500)"
nnz,644032
Density,1.5824951317349453e-05
Read-only,True
Size,19.7M
Storage ratio,0.00


Apply the interpolation with a sparse matrix multiplication:

In [15]:
%%time
regridded = ds.map(
    lambda var: xr.dot(
        weights.chunk({"cells": 2000000}).variable, var, dim=["x", "y"]
    ).assign_coords(weights.dggs.coord.coords)
)
regridded

CPU times: user 128 ms, sys: 1.75 s, total: 1.87 s
Wall time: 1.72 s


<xarray.Dataset> Size: 7MB
Dimensions:   (cells: 162789)
Coordinates:
  * cell_ids  (cells) uint64 1MB 926644821885 926644821886 ... 926656528545
Dimensions without coordinates: cells
Data variables:
    b02       (cells) float64 1MB dask.array<chunksize=(162789,), meta=sparse.numba_backend._coo.core.COO>
    b03       (cells) float64 1MB dask.array<chunksize=(162789,), meta=sparse.numba_backend._coo.core.COO>
    b04       (cells) float64 1MB dask.array<chunksize=(162789,), meta=sparse.numba_backend._coo.core.COO>
    b08       (cells) float64 1MB dask.array<chunksize=(162789,), meta=sparse.numba_backend._coo.core.COO>
Indexes:
    cell_ids  HealpixIndex(level=19, indexing_scheme=nested, kind=pandas)

Actually compute the results:

In [16]:
%%time
computed = regridded.compute().map(lambda var: var.copy(data=var.data.todense()))
computed

/home/jovyan/legacy-converters/.pixi/envs/dev/lib/python3.13/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 19.67 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: user 119 ms, sys: 513 ms, total: 633 ms
Wall time: 2.01 s


<xarray.Dataset> Size: 7MB
Dimensions:   (cells: 162789)
Coordinates:
  * cell_ids  (cells) uint64 1MB 926644821885 926644821886 ... 926656528545
Dimensions without coordinates: cells
Data variables:
    b02       (cells) float64 1MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    b03       (cells) float64 1MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    b04       (cells) float64 1MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    b08       (cells) float64 1MB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Indexes:
    cell_ids  HealpixIndex(level=19, indexing_scheme=nested, kind=pandas)

Display the result:

Note: `xdggs` does not yet support ellipsoidal coordinates, hence the shift compared to the base map

In [17]:
computed["b03"].dggs.explore(alpha=0.8)

Map(basemap=MaplibreBasemap(), controls=(FullscreenControl(), NavigationControl(), ScaleControl()), custom_att…